# ACETONE tutorial #2

**Implementing and using other versions of a layer**

Efficiency is a key aspect in the embedded sector, with each code being specifically adpated to a terget. As such, we need to be able to chose the implementation of each layer.

In this notebook, we'll explain how to create and use specific versions of a layer in ACETONE.

* When running this notebook on Colab, we need to install ACETONE 
* If you run this notebook locally, run it in the environment in which you installed ACETONE

In [ ]:
# TODO Installs on collab

In [ ]:
# Cleaning the working environment
from pathlib import Path
from os import remove, listdir

# Path to the example files
PATH_DIR = Path("../tests/models/lenet5/lenet5_trained")

# Path to generated directories
indirect_gemm_output_path = Path("demo_lenet_indirect_gemm")
std_gemm_output_path = Path("demo_lenet_std_gemm")
demo_output_path = Path("demo_lenet_optimized")

files_directories = [indirect_gemm_output_path, std_gemm_output_path, demo_output_path]

for directory in files_directories:
    if directory.exists():
        for file in listdir(directory):
            remove(directory / file)

## Imports

In this notebook, we'll use as an example a simple Lenet5 model exported to Keras' format h5. The used dataset is randomly generated for testing purposes.

![lenet5](./data/lenet5_trained.png)

In [ ]:
import numpy as np

from acetone_nnet import CodeGenerator, cli_compare, list_all_implementations, conv2d_factory
from acetone_nnet.generator import Conv2D

In [ ]:
model_path = PATH_DIR / "lenet5_trained.h5"
test_dataset = PATH_DIR / "test_input_lenet5.txt"
function_name = "demo_lenet"
nb_tests = 1

## Using ACETONE's native implementations

The framework laready provides, for some layers, several versions from which to choose before generating our code. 
In this notebook, we will focus on the convolution layer.

In [ ]:
implemented = list_all_implementations()
for layer_name in implemented:
    print(layer_name,":")
    for implementation in implemented[layer_name]:
        print("   ", implementation)
    print("\n")

We can change the implementation of a specific type of layer by using the class **CodeGenerator**'s argument `versions`. 

This argument takes a dictionnary containing a reference to the layer (usually the name) as key and the verion's name as value.

In this example, we want to use the algorithm `indirect_gemm_nn` to compute the convolution. ***(Describe algo)***

In [ ]:
# Version of the layer to use
conv_algorithm = "indirect_gemm_nn"

# Create an ACETONE CodeGenerator from the model
indirect_gemm_generator = CodeGenerator(file=model_path,
                                            function_name=function_name,
                                            test_dataset=test_dataset,
                                            versions={"Conv2D":conv_algorithm},
                                            nb_tests=nb_tests)

Once the generator has been created, we can generate the corresponding C code and compute the inference.

In [ ]:
indirect_gemm_generator.generate_c_files(indirect_gemm_output_path)
indirect_gemm_generator.compute_inference(indirect_gemm_output_path)

In [ ]:
# Version of the layer to use
conv_algorithm = "std_gemm_nn"

# Create an ACETONE CodeGenerator from the model
std_gemm_generator = CodeGenerator(file=model_path,
                                    function_name=function_name,
                                    test_dataset=test_dataset,
                                    versions={1:conv_algorithm, 3:conv_algorithm},
                                    nb_tests=nb_tests)

In [ ]:
std_gemm_generator.generate_c_files(std_gemm_output_path)
std_gemm_generator.compute_inference(std_gemm_output_path)

In [ ]:
# Compiling the code
! make -C demo_lenet_indirect_gemm all

# Running the executable
! ./demo_lenet_indirect_gemm/demo_lenet ./demo_lenet_indirect_gemm/output_c.txt

In [ ]:
# Compiling the code
! make -C demo_lenet_std_gemm all

# Running the executable
! ./demo_lenet_std_gemm/demo_lenet ./demo_lenet_std_gemm/output_c.txt

In [ ]:
cli_compare(reference_file=(indirect_gemm_output_path / "output_c.txt"), c_file=(std_gemm_output_path / "output_c.txt"), nb_tests=1)

## Adding a new implementation

Let's now assume that, after studies and tests, we have found a new way to perform a convolution : setting each element of the output to `0.42`.

This method being far more efficient and simple than any other, we want to use it with ACETONE. But, sadly, the framework doesn't have an implementation for it, we have to add it ourselves.

In [ ]:
# Printing all the algorithm implemented in ACETONE for a convolution
print("Base implementations : ")
print(conv2d_factory.list_implementations)

To implement it, we have to  create a new class inheriting from the `Conv2D` class (or one of its child classes). 

* The first method we must implement is called `generate_inference_code`. This method will construct the C code correponding to the layer, and return it as a string.
* The second method, `forwad_path_layer`, is optional. It tell the framework how to compute the output of the layer unsing Pyhton. If not given, the method defined in the parent class is used.



In [ ]:
# Creating a new implementation
class Conv2D_Demo(Conv2D):

    def __init__(self, **kwargs: int) -> None:
        """Build a Convolution layer with a demo implementation."""
        super().__init__(**kwargs)
    
    def generate_inference_code_layer(self) -> str:
        """Generate computation code for layer."""
        input_str = [prev_layer.output_str for prev_layer in self.previous_layer]
        ouptut_str = f"output_{self.path}"

        code_str =  f"    // {self.name}_{self.idx}\n    for (k = 0; k < {self.size}; ++k) {ouptut_str}[k] = 0.42;"
        return code_str
    
    def forward_path_layer(self, input_array) -> np.ndarray:
        return 0.42*np.ones((1,self.output_channels,self.output_height,self.output_width))

When parsing the neural network, each time ACETONE encounters a layer having several versions, it places a temporary layers. Once the model completly extracted, those placeholders are then replaced by a defintive layer whith the correct implementation, simply by extracting the values stored (such as weight, size, biases, ...) and using them to initialize a new layer.

In [ ]:
# Creating a Conv2D_Demo layer using the attributes of old_layer
def conv2d_demo_implementation(
        old_layer: Conv2D,
        conv_algo: str,
) -> Conv2D_Demo:
    return Conv2D_Demo(
        idx=old_layer.idx,
        conv_algorithm=conv_algo,
        size=old_layer.size,
        padding=old_layer.padding,
        strides=old_layer.strides,
        kernel_h=old_layer.kernel_h,
        kernel_w=old_layer.kernel_w,
        dilation_rate=old_layer.dilation_rate,
        nb_filters=old_layer.nb_filters,
        input_shape=[1, old_layer.input_channels, old_layer.input_height, old_layer.input_width],
        output_shape=[1, old_layer.output_channels, old_layer.output_height, old_layer.output_width],
        weights=old_layer.weights,
        biases=old_layer.biases,
        activation_function=old_layer.activation_function,
    )

Finally, to add the newly created implementation to ACETONE, we need to register it within the layer's version manager.

In [ ]:
conv2d_factory.register_implementation("demo", conv2d_demo_implementation)

print("Updated implementations : ")
print(conv2d_factory.list_implementations)

The new version being available in the list of implementations, we can now use it to generate code.

In [ ]:
# Version of the layer to use
conv_algorithm = "demo"

# Create an ACETONE CodeGenerator from the model
demo_generator = CodeGenerator(file=model_path,
                                    function_name=function_name,
                                    test_dataset=test_dataset,
                                    versions={"Conv2D":conv_algorithm},
                                    nb_tests=nb_tests)

demo_generator.generate_c_files(demo_output_path)

The code then has the optimized implementation and is ready to be deployed on any target !